In [25]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'1.14.1-dev20190507'

In [3]:
tf.enable_eager_execution()

In [4]:
tf.executing_eagerly()

True

In [5]:
from keras import bert

In [6]:
bert_layer = bert.BERT()

W0513 12:29:48.847255 140357207263040 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/keras/embeddings.py:45: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.



In [7]:
inputs = {'segment_ids': tf.constant([[0, 0]]), 'pos_ids': tf.constant([[0, 1]]), 'token_ids': tf.constant([[1, 4]])}

In [16]:
mask = tf.constant([[1, 0]])

In [14]:
model = tf.keras.Sequential([bert_layer])

In [17]:
r = model(inputs=inputs, mask=mask)

In [21]:
tf.keras.utils.get_file(
    fname='bert',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

666148864/666147468 [==============================] - 13s 0us/step


'./models/bert'

In [22]:
init_checkpoint = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
var_names = tf.train.list_variables(init_checkpoint)
check_point = tf.train.load_checkpoint(init_checkpoint)

In [34]:
model_vars = []
for v in model.variables:
    model_vars.append((v.name, v.shape))

In [35]:
model_vars

[('bert/embeddings/token_type_embeddings/embeddings:0',
  TensorShape([Dimension(2), Dimension(768)])),
 ('bert/embeddings/position_embeddings/embeddings:0',
  TensorShape([Dimension(512), Dimension(768)])),
 ('bert/embeddings/word_embeddings/embeddings:0',
  TensorShape([Dimension(119547), Dimension(768)])),
 ('bert/embeddings/LayerNorm/gamma:0', TensorShape([Dimension(768)])),
 ('bert/embeddings/LayerNorm/beta:0', TensorShape([Dimension(768)])),
 ('bert/encoder/layer_0/attention/self/query/kernel:0',
  TensorShape([Dimension(768), Dimension(768)])),
 ('bert/encoder/layer_0/attention/self/query/bias:0',
  TensorShape([Dimension(768)])),
 ('bert/encoder/layer_0/attention/self/key/kernel:0',
  TensorShape([Dimension(768), Dimension(768)])),
 ('bert/encoder/layer_0/attention/self/key/bias:0',
  TensorShape([Dimension(768)])),
 ('bert/encoder/layer_0/attention/self/value/kernel:0',
  TensorShape([Dimension(768), Dimension(768)])),
 ('bert/encoder/layer_0/attention/self/value/bias:0',
  Te

In [36]:
var_names

[('bert/embeddings/LayerNorm/beta', [768]),
 ('bert/embeddings/LayerNorm/gamma', [768]),
 ('bert/embeddings/position_embeddings', [512, 768]),
 ('bert/embeddings/position_embeddings/AdamWeightDecayOptimizer', [512, 768]),
 ('bert/embeddings/position_embeddings/AdamWeightDecayOptimizer_1',
  [512, 768]),
 ('bert/embeddings/token_type_embeddings', [2, 768]),
 ('bert/embeddings/word_embeddings', [119547, 768]),
 ('bert/encoder/layer_0/attention/output/LayerNorm/beta', [768]),
 ('bert/encoder/layer_0/attention/output/LayerNorm/gamma', [768]),
 ('bert/encoder/layer_0/attention/output/dense/bias', [768]),
 ('bert/encoder/layer_0/attention/output/dense/kernel', [768, 768]),
 ('bert/encoder/layer_0/attention/self/key/bias', [768]),
 ('bert/encoder/layer_0/attention/self/key/kernel', [768, 768]),
 ('bert/encoder/layer_0/attention/self/query/bias', [768]),
 ('bert/encoder/layer_0/attention/self/query/kernel', [768, 768]),
 ('bert/encoder/layer_0/attention/self/value/bias', [768]),
 ('bert/encode

In [32]:
model.load_weights(init_checkpoint)

ValueError: Shapes (768, 768) and (3072, 768) are incompatible